In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [7]:
%env TFHUB_CACHE_DIR=../tfhub

env: TFHUB_CACHE_DIR=../tfhub


In [20]:
import re
import numpy as np
import pandas as pd

from tqdm import tqdm
from IPython.core.display import display, HTML

from encoders.conveRT import conv_vec
from src.attention_search import attention_search

# Examples

Create a small search index locally

In [27]:
roster98 = [
    ["22","Keith Booth","SF","6-6","226","October 9","1974","us","R","Maryland"],
    ["1","Randy Brown","PG","6-2","190","May 22","1968","us","6","Houston, New Mexico State"],
    ["30","Jud Buechler","SF","6-6","220","June 19","1968","us","7","Arizona"],
    ["24","Scott Burrell","SF","6-7","218","January 12","1971","us","4","UConn"],
    ["35","Jason Caffey","PF","6-8","255","June 12","1973","us","2","Alabama"],
    ["9","Ron Harper","PG","6-6","185","January 20","1964","us","11","Miami University"],
    ["23","Michael Jordan","SG","6-6","195","February 17","1963","us","12","UNC"],
    ["25","Steve Kerr","PG","6-3","175","September 27","1965","lb","9","Arizona"],
    ["53","Joe Kleine","C","6-11","255","January 4","1962","us","12","Notre Dame, Arkansas"],
    ["7","Toni Kukoč","SF","6-10","192","September 18","1968","hr","4,"],
    ["5","Rusty LaRue","PG","6-2","210","December 10","1973","us","R","Wake Forest"],
    ["13","Luc Longley","C","7-2","265","January 19","1969","au","6","New Mexico"],
    ["33","Scottie Pippen","SF","6-8","210","September 25","1965","us","10","University of Central Arkansas"],
    ["91","Dennis Rodman","PF","6-7","210","May 13","1961","us","11","Southeastern Oklahoma State University"],
    ["8","Dickey Simpkins","PF","6-9","248","April 6","1972","us","3","Providence"],
    ["42","David Vaughn","PF","6-9","240","March 23","1973", "us","2","Memphis"],
    ["34","Bill Wennington","C","7-0","245","April 26","1963","ca","10","St. John's"]
]

bulls_df = pd.DataFrame(roster98)
bulls_df

,0,1,2,3,4,5,6,7,8,9
0,22,Keith Booth,SF,6-6,226,October 9,1974,us,R,Maryland
1,1,Randy Brown,PG,6-2,190,May 22,1968,us,6,"Houston, New Mexico State"
2,30,Jud Buechler,SF,6-6,220,June 19,1968,us,7,Arizona
3,24,Scott Burrell,SF,6-7,218,January 12,1971,us,4,UConn
4,35,Jason Caffey,PF,6-8,255,June 12,1973,us,2,Alabama
5,9,Ron Harper,PG,6-6,185,January 20,1964,us,11,Miami University
6,23,Michael Jordan,SG,6-6,195,February 17,1963,us,12,UNC
7,25,Steve Kerr,PG,6-3,175,September 27,1965,lb,9,Arizona
8,53,Joe Kleine,C,6-11,255,January 4,1962,us,12,"Notre Dame, Arkansas"
9,7,Toni Kukoč,SF,6-10,192,September 18,1968,hr,"4,",None


BYOE (bring your own embeddings). For this demo, we'll apply [ConveRT](https://github.com/PolyAI-LDN/polyai-models#convert) from PolyAI to create a sequence of embeddings from our search index

In [60]:
roster_list = bulls_df.to_numpy().tolist()
roster_enc = np.concatenate([conv_vec(f"{player}") for player in tqdm(roster_list)])

100%|██████████| 17/17 [00:00<00:00, 18.96it/s]


Next, we can encode a query to perform a simple search:

In [220]:
query = conv_vec("mexico us")
search_index = roster_enc
search_values = roster98

results = attention_search(query, search_index, search_values, verbose=True)
pd.DataFrame(results)

Searched 17 records in 0.0002 seconds


,0,1,2,3,4,5,6,7,8,9
0,1,Randy Brown,PG,6-2,190,May 22,1968,us,6,"Houston, New Mexico State"
1,13,Luc Longley,C,7-2,265,January 19,1969,au,6,New Mexico
2,91,Dennis Rodman,PF,6-7,210,May 13,1961,us,11,Southeastern Oklahoma State University


Let's try searching a larger dataset like [Amazon's Fine Food Reviews](https://www.kaggle.com/snap/amazon-fine-food-reviews?select=Reviews.csv)

In [67]:
amazon_food_reviews = pd.read_csv("data/Reviews.csv")
text_only = amazon_food_reviews["Text"]
text_only.shape

(568454,)

Just like on the baking shows, I've already encoded the first 100,000 records 

In [10]:
# uncomment this block re-encode (this can be instensive on a CPU)
# res = []
# for item in tqdm(text_only[:100000]):
#     res.append(conv_vec(item))
# amazon_fine_food_vec = np.concatenate(res)
# np.save("amazon_fine_food", amazon_fine_food_vec)

In [65]:
amazon_fine_food_vec = np.load("data/amazon_fine_food.npy")
amazon_fine_food_vec.shape

(100000, 1024)

In [312]:
results = attention_search(
    conv_vec("ramen noodles"), 
    amazon_fine_food_vec, 
    text_only.to_list()[:100000], # encoded only the first 100,000
    n_results=5,
    verbose=True
)

Searched 100000 records in 0.0264 seconds


In [313]:
for i, r in enumerate(results):
    if not re.findall(r"\*{2,}|-{2,}", r): # ignore messy records  
        # r = re.sub(r"(ramen|noodles)", r"<mark><b><u>\1</u></b></mark>", r)
        display(HTML(f"<hr /> {r[:300]}..."))

# Benchmarking

In [77]:
# TODO: https://github.com/erikbern/ann-benchmarks